In [5]:
import keras_tuner
from tensorflow import keras
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.densenet import DenseNet121
from keras.layers import BatchNormalization, Activation, Dropout, MaxPooling2D,Conv2D
import pandas as pd

In [6]:
size = 96


In [9]:
#EmotioNet
EmotioNet_Emotion_X_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_train")
EmotioNet_Emotion_y_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_Y_train")
EmotioNet_Emotion_X_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_val")
EmotioNet_Emotion_y_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_Y_val")
X_train = EmotioNet_Emotion_X_train
X_val = EmotioNet_Emotion_X_val
y_train = EmotioNet_Emotion_y_train
y_val = EmotioNet_Emotion_y_val

In [10]:
from skimage.transform import resize
def resize_images(images):
    resized_images = []
    for image in images:
        resized_images.append(resize(image, (size, size)))
    return np.asarray(resized_images).astype('float32')

In [11]:
X_train = resize_images(X_train)
X_val = resize_images(X_val)
#X_test = resize_images(X_val)

In [12]:
X_train = np.resize(X_train,(len(X_train),size,size))
X_val = np.resize(X_val,(len(X_val),size,size))

In [13]:
X_train = np.resize(X_train,(len(X_train),size,size,1))
X_val = np.resize(X_val,(len(X_val),size,size,1))
#X_test = np.resize(X_test,(len(X_test),size,size,1))

In [14]:
print(np.shape(X_train))
print(np.shape(X_val))
#print(np.shape(X_test))

(877, 96, 96, 1)
(110, 96, 96, 1)


In [15]:
y_train

216    3.0
512    5.0
175    3.0
594    4.0
300    5.0
      ... 
743    2.0
263    3.0
786    5.0
462    5.0
669    5.0
Name: emotion, Length: 877, dtype: float64

In [16]:
from sklearn.preprocessing import OneHotEncoder

In [17]:
y_train = np.array(np.array(y_train).reshape(-1, 1))
y_val = np.array(np.array(y_val).reshape(-1, 1))
#y_test = np.array(np.array(y_test).reshape(-1, 1))

In [18]:
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder.fit_transform(y_train)
onehot_encoder2 = OneHotEncoder(sparse_output=False)
y_val = onehot_encoder2.fit_transform(y_val)
#y_test = onehot_encoder2.fit_transform(y_test)

In [19]:
np.shape(y_val)

(110, 6)

In [20]:
np.shape(y_train)

(877, 6)

In [21]:
NUM_EPOCHS = 20

In [22]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model

In [23]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

2023-04-14 11:11:00.144363: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 11:11:00.345003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 11:11:00.346183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

['input_1', 'zero_padding2d', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_1', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'conv

In [24]:
num_classes = 6

In [25]:
precision = keras.metrics.Precision(name="Precision")
recall = keras.metrics.Recall(name="Recall")

In [26]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)


In [27]:
import datetime
folder_name = 'logs/DenseNet/Tuning/Emotion/'
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [28]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Tuning/DenseNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [47]:
def build_model(hp):
    lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
    l1 = 0
    l2 = hp.Choice('l2',  values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_hidden = hp.Int('num_hidden', 32,1024,32)
    regularizer = tf.keras.regularizers.l1_l2(l1, l2)
    model = Sequential()
    model.add(densenet_updated)
    model.add(Flatten())
    model.add(Dense(num_hidden, kernel_regularizer=regularizer))
    model.add(BatchNormalization(scale=False,
                                    center=False))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss='categorical_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])
    return model


In [48]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_custom_f1', 'max'),
    max_trials = 20,
    overwrite=True
)

In [49]:
tuner.search(
    X_train, y_train, validation_data=(X_val, y_val),
    epochs=20
)

Trial 20 Complete [00h 00m 45s]
val_custom_f1: 0.7927413582801819

Best val_custom_f1 So Far: 0.818206250667572
Total elapsed time: 00h 15m 20s
INFO:tensorflow:Oracle triggered exit


In [50]:
topN = 2
for x in range(topN):
    print(tuner.get_best_hyperparameters(topN)[x].values)
    print(tuner.get_best_models(topN)[x].summary())

{'lrate': 0.0002761381931692287, 'l2': 0.0, 'num_hidden': 416}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 3, 3, 1024)        7031232   
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 416)               3834272   
                                                                 
 batch_normalization (BatchN  (None, 416)              832       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 416)               0         
                                                                 
 dropout (Dropout)           (None, 416)               0   